# Disease-state discovery on HLCA - prepare datasets

In this notebook we compare reference designs for disease-state identification on the dataset from [Adams et al](https://www.science.org/doi/10.1126/sciadv.aba1983), using trained models from the [Human Lung Cell Atlas](https://www.biorxiv.org/content/10.1101/2022.03.10.483747v1.full). 

- Model repository: https://zenodo.org/record/6337966/#.Yzv65ezMK3J
- Extended HLCA data objects: https://beta.fastgenomics.org/datasets/detail-dataset-427f1eee6dd44f50bae1ab13f0f3c6a9#Files


In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

import oor_benchmark

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
## r2py setup
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)


In [4]:
%load_ext rpy2.ipython

In [5]:
%%R
library(tidyverse)
library(patchwork)

remove_x_axis <- function(){
  theme(axis.text.x = element_blank(), axis.ticks.x = element_blank(), axis.title.x = element_blank())  
}

remove_y_axis <- function(){
  theme(axis.text.y = element_blank(), axis.ticks.y = element_blank(), axis.title.y = element_blank())  
}

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to create bus connection: Host is down


## Download data and trained models 

From Zenodo repository

In [3]:
%%bash
cd /lustre/scratch117/cellgen/team205/ed6/HLCA/

## Download extended HLCA embeddings 
if ! [ -d Kaminski_2020_emb_LCAv2.h5ad ];
    then echo "Downloading scArches embeddings from zenodo"
    wget -nv https://zenodo.org/record/6337966/files/HLCA_extended_models_and_embs.zip
    unzip HLCA_extended_models_and_embs.zip
    rm HLCA_extended_models_and_embs.zip
fi

2022-10-04 09:55:03 URL:https://zenodo.org/record/6337966/files/HLCA_extended_models_and_embs.zip [380368487/380368487] -> "HLCA_extended_models_and_embs.zip" [1]


Archive:  HLCA_extended_models_and_embs.zip
   creating: HLCA_extended_models_and_embs/
   creating: HLCA_extended_models_and_embs/surgery_output_embeddings/
   creating: HLCA_extended_models_and_embs/surgery_models/
  inflating: HLCA_extended_models_and_embs/surgery_output_embeddings/KULeuven_Thienpont_2018Lambrechts_v2_emb_LCAv2.h5ad  
  inflating: HLCA_extended_models_and_embs/surgery_output_embeddings/Sims_2019_emb_LCAv2.h5ad  
  inflating: HLCA_extended_models_and_embs/surgery_output_embeddings/Lafyatis_Rojas_2019_disease_emb_LCAv2.h5ad  
  inflating: HLCA_extended_models_and_embs/surgery_output_embeddings/Budinger_2021_emb_LCAv2.h5ad  
  inflating: HLCA_extended_models_and_embs/surgery_output_embeddings/Shalek_2018_emb_LCAv2.h5ad  
  inflating: HLCA_extended_models_and_embs/surgery_output_embeddings/Wunderink_2021_cryo_emb_LCAv2.h5ad  
  inflating: HLCA_extended_models_and_embs/surgery_output_embeddings/Eils_2020_emb_LCAv2.h5ad  
  inflating: HLCA_extended_models_and_embs/surgery

## Prepare ACR design

Use control and condition mapped to HLCA core (from original pub). Full gene expression matrix was downloaded from the original publication.

In [ ]:
# Objects processed by Amanda
datadir = '/lustre/scratch117/cellgen/team205/ed6/HLCA/'
h5ad_file = '/nfs/team205/ao15/Adult_lung_MadissoonOliver/Kaminski2020_COPD/Kaminski_2020_HLCAlabeltransfer.h5ad'
adata = sc.read_h5ad(h5ad_file, backed=False)
adata_hlca = sc.read_h5ad(datadir + "Kaminski_2020_emb_LCAv2.annotated.h5ad", backed=True)
## Keep cells analysed in HLCA
adata = adata[adata_hlca.obs_names.str.strip("_adams")].copy()
adata.obsm = adata_hlca.obsm.copy()
adata.obsp = adata_hlca.obsp.copy()
adata.write_h5ad(datadir + "Kaminski_2020_emb_LCAv2.annotated.with_counts.h5ad")

In [73]:
datadir = '/lustre/scratch117/cellgen/team205/ed6/HLCA/'
adata = sc.read_h5ad(datadir + "Kaminski_2020_emb_LCAv2.annotated.with_counts.h5ad")

## Set IDs to var_names
adata.var['gene_name'] = adata.var.index.values
adata.var_names = adata.var['Ensembl_GeneID']

## Renamer
rename_obs_cols = {
    'Library_Identity':'sample',
    'Subject_Identity':'individual', 
    'Disease_Identity':'disease'
}

adata.obs = adata.obs.rename(rename_obs_cols, axis=1)

## Get label uncertainty from original HLCA study
full_file = '/lustre/scratch117/cellgen/team205/ed6/HLCA/HLCA_v1_extended_raw_counts_2000hvgs.h5ad' ## Downloaded from FastGenomics
full_adata = sc.read_h5ad(full_file, backed='r')

adata.obs['HLCA_label_transfer_uncertainty']  = full_adata.obs.loc[adata.obs_names + "_adams"]['ext_transf_uncert_level_5'].values

## Add labels to train scANVI model for CR design
adata.obs['scanvi_labels'] = full_adata.obs.loc[adata.obs_names + "_adams"]['original_ann_level_2'].values

Save ACR design object

In [79]:
adata.write_h5ad(data_dir + "Kaminski_2020_oor_design.ACR.h5ad")

In [44]:
full_adatair = '/lustre/scratch117/cellgen/team205/ed6/HLCA/'
# emb_file = '/lustre/scratch117/cellgen/team205/ed6/HLCA/HLCA_extended_models_and_embs/surgery_output_embeddings/Kaminski_2020_emb_LCAv2.h5ad'
# full_file = '/lustre/scratch117/cellgen/team205/ed6/HLCA/HLCA_v1_extended_raw_counts_2000hvgs.h5ad' ## Downloaded from FastGenomics
# full_adata = sc.read_h5ad(full_file, backed='r')

# query_adata = sc.read_h5ad(emb_file)

# ## Get obs for Kaminski_2020
# query_adata.obs = full_adata.obs[full_adata.obs_names.isin(query_adata.obs_names)].copy()

# ## Exclude cols that are all NAs for this dataset
# non_na_obs_columns = query_adata.obs.columns[
#     ((query_adata.obs == 'nan').sum() < query_adata.n_obs) &
#     (query_adata.obs.isna().sum() < query_adata.n_obs)
# ].tolist()

# ## Remove obs that are the same for all cells
# ## (move to uns)
# dataset_meta = {}
# keep_obs_columns = []
# for col in non_na_obs_columns:
#     if query_adata.obs[col].unique().shape[0] == 1:
#         dataset_meta[col] = query_adata.obs[col].unique()[0]
#     else:
#         keep_obs_columns.append(col)
        
# ## Remove HLCA analysis columns
# keep_obs_columns = [x for x in keep_obs_columns if not x.startswith('ext_transf')]

# ## Keep column for label transfer uncertainty score used in HLCA ms
# keep_obs_columns.append('ext_transf_uncert_level_5')


# query_adata.obs = query_adata.obs[keep_obs_columns].copy()
# for c in query_adata.obs.columns:
#     if query_adata.obs[c].dtype == 'category':
#         query_adata.obs[c] = query_adata.obs[c].cat.remove_unused_categories()
# query_adata.obsm['X_scVI'] = query_adata.X.copy()
# query_adata.uns['dataset_metadata'] = dataset_meta.copy()

# ## Renamer
# rename_obs_cols = {
#     'subject_ID':'individual', 
#     'condition':'disease', 
#     'ext_transf_uncert_level_5':'HLCA_label_transfer_uncertainty'
# }

# query_adata.obs = query_adata.obs.rename(rename_obs_cols, axis=1)

# ## Read gene expression matrix (downloaded from original pub)
# adata_gex = sc.read_h5ad('/nfs/team205/ao15/Adult_lung_MadissoonOliver/Kaminski2020_COPD/Kaminski_2020_HLCAlabeltransfer.h5ad')

# query_adata.X = adata_gex[query_adata.obs_names.str.strip("_adams")].X.copy()
# query_adata.obs['Celltype_HLCA'] = adata_gex[query_adata.obs_names.str.strip("_adams")].obs['Celltype_HLCA'].values

## Prepare AR design

In [6]:
data_dir = '/lustre/scratch117/cellgen/team205/ed6/HLCA/'
acr_adata = sc.read_h5ad(data_dir + "Kaminski_2020_oor_design.ACR.h5ad", backed=True)

In [7]:
full_file = '/lustre/scratch117/cellgen/team205/ed6/HLCA/HLCA_v1_extended_raw_counts_2000hvgs.h5ad' ## Downloaded from FastGenomics
full_adata = sc.read_h5ad(full_file, backed='r')

In [8]:
## Subset to core atlas and Kaminski
keep_obs = full_adata.obs_names[(full_adata.obs['HLCA_core_or_extension'] == 'core') | (full_adata.obs['study'] == 'Kaminski_2020')]
adata = full_adata[keep_obs].to_memory()

## Renamer
rename_obs_cols = {
    'subject_ID':'individual', 
    'condition':'disease', 
    'ext_transf_uncert_level_5':'HLCA_label_transfer_uncertainty'
}

adata.obs = adata.obs.rename(rename_obs_cols, axis=1)

In [17]:
## Store annotation
adata.obs['Celltype_HLCA'] = np.nan
adata.obs.loc[adata.obs['HLCA_core_or_extension'] == 'core', 'Celltype_HLCA'] = adata.obs.loc[adata.obs['HLCA_core_or_extension'] == 'core', 'ann_finest_level']

kaminski_cells = adata.obs_names[adata.obs['HLCA_core_or_extension'] != 'core']
adata.obs.loc[kaminski_cells, 'Celltype_HLCA'] = acr_adata.obs.loc[kaminski_cells.str.strip("_adams"),'Celltype_HLCA'].values

In [18]:
keep_obs = ['individual', 'sample', 'disease', 'study', 'HLCA_core_or_extension', 'Celltype_HLCA', 'HLCA_label_transfer_uncertainty']

In [19]:
adata.obs = adata.obs[keep_obs]

In [20]:
## Store joint embedding
emb_adata = sc.read_h5ad(data_dir + "HLCA_emb_and_metadata.h5ad", backed=False)

X_scVI_core = emb_adata[adata.obs_names[adata.obs['HLCA_core_or_extension'] == 'core']].X.copy()
X_scVI_kaminski = acr_adata[kaminski_cells.str.strip("_adams")].obsm['X_scVI'].copy()

assert all(np.hstack([adata.obs_names[adata.obs['HLCA_core_or_extension'] == 'core'], kaminski_cells]) == adata.obs_names)
adata.obsm['X_scVI'] = np.vstack([X_scVI_core, X_scVI_kaminski])

In [21]:
adata

AnnData object with n_obs × n_vars = 892534 × 2000
    obs: 'individual', 'sample', 'disease', 'study', 'HLCA_core_or_extension', 'Celltype_HLCA', 'HLCA_label_transfer_uncertainty'
    uns: 'ann_finest_level_colors', 'ann_level_1_core_transferred_colors'
    obsm: 'X_umap', 'X_scVI'

In [30]:
## Clean disease assingment
adata.obs['disease'] = np.where(adata.obs['disease'].isin(['healthy', 'nan', 'Control', "Healthy", 'had TB as a child (fully treated over 30+ years)']), 'Control', adata.obs['disease'])
adata.obs['disease'] = adata.obs['disease'].astype("category").cat.reorder_categories(
    ['non-small cell lung cancer', 'carcinoid','worsening respiratory function prior to arrest', "Control",'COPD', 'IPF'])

In [31]:
adata.write_h5ad(data_dir + "Kaminski_2020_oor_design.AR.h5ad")

In [52]:
n_donors = acr_adata.obs[['disease', 'individual']].drop_duplicates().value_counts('disease')
n_cells = acr_adata.obs[['disease', 'individual']].value_counts('disease')
df = pd.concat([n_donors, n_cells], 1)
df.columns = ['n_donors', 'n_cells']
print(df)

         n_donors  n_cells
disease                   
IPF            32   144404
Control        28    95303
COPD           18    67943


/tmp/ipykernel_51420/1809269403.py:3: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df = pd.concat([n_donors, n_cells], 1)


In [41]:
print("# cells")


# cells


disease
IPF        144404
Control     95303
COPD        67943
dtype: int64

## Prep CR design

In [158]:
cr_adata = sc.read_h5ad(data_dir + "Kaminski_2020_oor_design.ACR.h5ad", backed=False)

In [171]:
%%bash
cd /lustre/scratch117/cellgen/team205/ed6/HLCA/
wget https://raw.githubusercontent.com/LungCellAtlas/HLCA_reproducibility/main/notebooks/3_atlas_extension/HLCA_extension_data_preprocessing/genes_for_mapping.csv

--2023-01-11 17:44:48--  https://raw.githubusercontent.com/LungCellAtlas/HLCA_reproducibility/main/notebooks/3_atlas_extension/HLCA_extension_data_preprocessing/genes_for_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45063 (44K) [text/plain]
Saving to: ‘genes_for_mapping.csv.1’

     0K .......... .......... .......... .......... ....      100% 15.7M=0.003s

2023-01-11 17:44:48 (15.7 MB/s) - ‘genes_for_mapping.csv.1’ saved [45063/45063]



In [179]:
## Use the same HVGs used in core
genes_for_mapping = pd.read_csv(data_dir+'genes_for_mapping.csv',index_col=0)
cr_adata.var['mapping_gene'] = cr_adata.var_names.isin(genes_for_mapping.index)

In [190]:
cr_adata.obs['dataset'] = 'Kaminski_2020'

In [215]:
cr_adata.write_h5ad(data_dir + "Kaminski_2020_oor_design.CR.h5ad")

We generate a common latent embedding between control and disease dataset running the script `train_CR_design.py` 